In [1]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder

In [2]:
import xgboost
from xgboost import XGBClassifier
xgboost.__version__

'1.7.4'

In [3]:
data = pd.read_csv('../Field_training_data_noOutliers_newContaminationMetric.csv')
labels = pd.read_csv('../Field_training_labels_noOutliers_newContaminationMetric.csv')
data = data.reset_index(drop=True)
labels = labels.reset_index(drop=True)

idx = labels.index[labels['Trophic mode'] == 'Un']
train_labels = labels.drop(idx)
train_data = data.drop(idx)

features = pd.read_csv('../Extracted_Pfams_noOutliers_newContaminationMetric_xg.csv')

alohaDiel_TPM_merged_lats = pd.read_csv('../../alohaDiel/alohaDiel_surface_tpm_updatedMarferret_marmicroDb2023_noOutliers_fixedTPM_fall2023.csv')

,PF00001,PF00002,PF00003,PF00004,PF00005,PF00006,PF00007,PF00008,PF00009,PF00010,...,PF20030,PF20043,PF20064,PF20071,PF20102,PF20133,PF20143,PF20144,PF20145,PF20146
0,0.00000,0.000000,0.000000,204.798787,963.273295,0.000000,0.0,3.990115,133.533699,0.000000,...,0.000000,0.0,0.000000,0.000000,43.673660,0.000000,0.000000,0.0,0.000000,0.0
1,0.00000,0.000000,1819.990148,900.199704,11897.805383,1060.917524,0.0,14.779051,743.668074,0.000000,...,129.759407,0.0,48.545587,3577.527561,0.000000,288.374509,0.000000,0.0,103.942217,0.0
2,152.57419,128.479234,81.759513,449.282127,1179.859501,1469.873941,0.0,557.732885,852.359134,925.613266,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,278.849969,0.0
3,0.00000,0.000000,0.000000,2273.539613,1178.426596,0.000000,0.0,0.000000,814.893793,17.567993,...,0.000000,0.0,0.000000,0.000000,0.000000,115.544396,0.000000,0.0,0.000000,0.0
4,0.00000,0.000000,0.000000,916.247038,2951.945616,451.965745,0.0,4.285275,1210.566586,0.000000,...,0.854560,0.0,0.000000,0.000000,5.029901,1.108547,9.422745,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588,0.00000,0.000000,4.538723,890.587633,948.009710,0.000000,0.0,16.172551,152.624527,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,231.343006,0.0
589,0.00000,0.000000,0.000000,410.257394,898.851905,48.140884,0.0,43.316538,188.876927,0.000000,...,0.000000,0.0,0.000000,0.000000,158.662610,0.000000,8.099434,0.0,0.000000,0.0
590,0.00000,0.000000,86.204478,2150.959825,4004.584858,153.723381,0.0,19.867670,1204.882222,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,32.119483,9.823981,0.0,184.027738,0.0
591,0.00000,0.000000,0.000000,2117.584643,3259.651915,963.512987,0.0,87.490472,2232.219331,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,31.110783,3.252044,0.0,45.917537,0.0


In [4]:
alohaDiel_TPM_merged_lats.columns[(alohaDiel_TPM_merged_lats == 0).all()]

Index(['PF03040', 'PF12329', 'PF14259'], dtype='object')

In [6]:
features = features['pfam']

In [13]:
len(features)

183

In [14]:
alohaDiel_TPM_merged_lats = alohaDiel_TPM_merged_lats[features]

In [16]:
alohaDiel_TPM_merged_lats.columns[(alohaDiel_TPM_merged_lats == 0).all()]

Index(['PF03040', 'PF14259', 'PF12329'], dtype='object')

In [17]:
train_data = train_data[features]

In [18]:
le = LabelEncoder()

In [19]:
X,y = train_data, le.fit_transform(train_labels['Trophic mode'])
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [20]:
alohaDiel = scaler.transform(alohaDiel_TPM_merged_lats)

In [21]:
#gamma: 0.5, learning_rate: 0.2, max_depth: 3, n_estimators: 100, reg_lambda: 0.0 
model = XGBClassifier(gamma=0.5, learning_rate=0.2, max_depth=3, n_estimators=100, reg_lambda=0.0)
model.fit(X,y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.5, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [22]:
xg_predictions_alohaDiel = model.predict(alohaDiel)

In [24]:
prob = model.predict_proba(alohaDiel)

In [25]:
alohaDiel_predictions = pd.DataFrame(data={'xg_pred':xg_predictions_alohaDiel})

In [26]:
probDF = pd.DataFrame(data={'probability':np.max(prob, axis = 1)})

In [28]:
alohaDiel_predictions.to_csv('../../alohaDiel/G1_diel_trophicModePredictions_updatedMarferret_marmicroDb2023_noOutliers_fall2023',index=False)

In [29]:
probDF.to_csv('../../alohaDiel/G1_diel_trophicModePredictionsProbabilities_updatedMarferret_marmicroDb2023_noOutliers_fall2023',index=False)